# 4. Woordfrequenties - hoe vaak komt een woord voor in een document of een corpus?

Welke woorden komen het meeste voor in een bepaald egodocument? De onderstaande code berekent de frequenties van alle woorden in een enkele tekst. De tekst waarin wordt gezocht wordt bepaald door de waarde van de variabele `egodocument`. De code toont vervolgens de 30 meest voorkomende woorden in deze tekst. De variabele `max` het aantal woorden dat wordt getoond. 

Ook hier is het van belang de stopwoordenlijst te gebruiken om alleen zinvolle frequenties in de resultaten terug te krijgen die te maken hebben met de inhoud, en om niet ter zake doende lidwoorden, bijwoorden en voorzetsels er uit te filteren. 

In [ ]:
from kitlvTdm import *
import os
from os.path import join
import re
import pandas as pd

df = pd.DataFrame(columns = ['word','frequentie', 'total_tokens'])

max = 150 # Het maximum aantal woorden dat later wordt geprint en opgeslagen
path = 'corpus' # Het pad naar de map met de corpus

min_year = 1930
max_year = 1960
outfile = f"corpus_frequency_{min_year}_{max_year}"


def sortedByValue( dict ):
    return sorted( dict , key=lambda x: dict[x], reverse = True)

# de totale frequenties worden opgeslagen in een dictionary
freqTotal = dict() 



# Hier worden het aantal woorden (tokens) en documenten bijgehouden
tokensTotal = 0
egodocuments = 0

for i, file in enumerate(os.listdir(path)):
    
    percentage_done = i / len(os.listdir(path)) * 100
    print("percentage done: {:.2f}%".format(percentage_done), end = "\r")
    
    if re.search( 'txt$' , file ):
        
        year = showYear(file)

        # Hier wordt gecheckt er een jaar van het boek bekent is en het jaar uit 4 tekens bestaat.
        if year != "" and len(year) == 4:
            year = int(year) # Zo ja: maak er een nummer ipv tekst van

            # Hier wordt gecheckt of het jaar van het boek in de periode valt.
            if year >= min_year and year <= max_year:
                egodocuments += 1

                # bereken per woord hoe vaak het voor komt
                freq = calculateWordFrequencies( join( path, file ) )

                # Haal de stopwoorden eruit
                freq = removeStopwords(freq)

                for word in freq:
                    if word  in freqTotal: # Als het woord al in freqTotal staat: tel extra aantal erbij op
                        freqTotal[word] += freq[word]
                    else: # anders, plaats het woord in freqTotal
                        freqTotal[word] = freq[word]

                tokensTotal += numberOfTokens( join( path, file ) )

                if i > 50:
                    break
            
for count, word in enumerate(sortedByValue(freqTotal)):
    print( f' { word } => { freqTotal[word] / tokensTotal }' )
    
    df = df.append({'word' : word, 'frequency' : freqTotal[word] / tokensTotal, 'total_tokens': tokensTotal}, ignore_index = True)
    
    if count == max:
        break
        
df.to_excel(f"{outfile}.xlsx")
print(f"Excel file saved as {outfile}.xlsx") 
